### 볼린저 밴드 함수화
1. 첫 번째 함수를 생성(매개변수 데이터프레임 하나)
    1. 결측치와 이상치를 제거한다
    2. 컬럼을 종가만 두고 모두 삭제
    3. 이동 평균선, 상단 밴드, 하단 밴드 파생변수 생성
2. 두 번째 함수를 생성(매개변수 데이터프레임 하나)
    1. 거래 내역이라는 파생 변수 생성 값은 ""
    2. 볼린저 밴드의 거래 내역을 추가
    3. 거래 내역을 출력(print)
3. 세 번째 함수를 생성(매개변수 데이터프레임 하나)
    1. 수익률 파생변수 생성 값은 "1"
    2. 판매를 한 날의 수익률 변경
    3. 누적 수익률을 계산하여 새로운 파생변수를 생성
    4. 최종 누적 수익률을 출력(print)

In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

In [16]:
def pricedf(df, column = 'Adj Close', start = '20110101', end = '20200101'):
    df = df.loc[~df.isin((np.nan, np.inf, -np.inf)).any(axis='columns')]
    price_df = df[['Adj Close']]
    price_df['center'] = price_df['Adj Close'].rolling(20).mean()

    price_df['ub'] = price_df['center'] + ( 2 * price_df['Adj Close'].rolling(20).std() )
    price_df['lb'] = price_df['center'] - ( 2 * price_df['Adj Close'].rolling(20).std() )
    return price_df

In [19]:
def tradedf(df):
    df['trade'] = ''
    for i in df.index:
    # 상단 밴드보다 종가가 높은 경우
        if df.loc[i, 'Adj Close'] > df.loc[i, 'ub']:
            # 현재 구매 상태이면
            if df.shift(1).loc[i, 'trade'] == 'buy':
                # 매도
                df.loc[i, 'trade'] = ''
            else:
                df.loc[i, 'trade'] = ''
        # 하단 밴드보다 종가가 낮은 경우
        elif df.loc[i,'Adj Close'] < df.loc[i,'lb']:
            # 현재 구매 상태이면
            if df.shift(1).loc[i, 'trade'] == 'buy':
                # 구매 상태를 유지
                df.loc[i, 'trade'] = 'buy'
            else:
                df.loc[i, 'trade'] = 'buy'
        else:
            # 현재 구매 상태면
            if df.shift(1).loc[i, 'trade'] == 'buy':
                # 구매 상태를 유지
                df.loc[i, 'trade'] = 'buy'
            else:
                df.loc[i, 'trade'] = ''
    return df

In [20]:
def rtndf(df):
    rtn = 1.0
    df['return'] = 1
    buy = 0.0
    sell = 0.0
    for i in df.index:
    # 구매가를 출력
        if (df.shift(1).loc[i, 'trade'] == '') and \
            (df.loc[i, 'trade'] == 'buy'):
            buy = df.loc[i, 'Adj Close']
        #    print('진입일 :', i, '구매 가격 :', buy)
        # 판매가를 출력
        elif (df.shift(1).loc[i, 'trade'] == 'buy') and \
            (df.loc[i, 'trade'] == ''):
            sell = df.loc[i, 'Adj Close']
            rtn = (sell - buy)/buy + 1
            df.loc[i, 'return'] = rtn
        #    print('판매일 :', i, '판매 가격 :', sell, '수익률 :', rtn)

        # 구매가, 판매가를 초기화
        if df.loc[i, 'trade'] == '':
            buy = 0.0
            sell = 0.0

    acc_rtn = 1.0
    for i in df.index:
        rtn = df.loc[i, 'return']
        acc_rtn *= rtn
        df.loc[i, 'acc_rtn'] = acc_rtn
    return df

    #print('누적 수익율 :', acc_rtn)

In [21]:
df = pd.read_csv("../csv/GM.csv",  index_col='Date')
df = pricedf(df,'Adj Close')

C:\Users\HangAri\AppData\Local\Temp\ipykernel_8992\94534201.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price_df['center'] = price_df['Adj Close'].rolling(20).mean()
C:\Users\HangAri\AppData\Local\Temp\ipykernel_8992\94534201.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price_df['ub'] = price_df['center'] + ( 2 * price_df['Adj Close'].rolling(20).std() )
C:\Users\HangAri\AppData\Local\Temp\ipykernel_8992\94534201.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

In [22]:
df = tradedf(df)
df['trade'].value_counts()

buy    1130
       1032
Name: trade, dtype: int64

In [23]:
df = rtndf(df)

In [30]:
print('누적 수익율 :', df['acc_rtn'].iloc[-1])

누적 수익율 : 1.1764314536564138
